# **Clone and download models**

In [ ]:
%%writefile requirements.txt
colabcode
pyngrok
fastapi
python-multipart

Writing requirements.txt


In [ ]:
%%capture
!git clone https://github.com/A40405/STTR2022.git
!git clone https://github.com/A40405/stablediffusion2022.git
!git clone https://github.com/A40405/ngamcuukhoahoc.git
!pip install -r requirements.txt
!pip install -r STTR2022/requirements.txt
!pip install -r stablediffusion2022/requirements.txt
del _

In [ ]:
%mkdir STTR2022/checkpoint_model -p
%mkdir  stablediffusion2022/checkpoints -p

import os
import gdown
url = 'https://drive.google.com/uc?id=1oNt0YwJvyNQfrTSr2V3A5ee6AIKqvmLp'
output = './STTR2022/checkpoint_model/checkpoint0005.pth'
if not os.path.isfile(output):
  gdown.download(url, output, quiet=False)

# url = 'https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-nonema-pruned.ckpt'
url = 'https://drive.google.com/uc?id=1pWOLw6ZiQJq0KkQyrZwNKeBVqEhWaamo'
output = './stablediffusion2022/checkpoints/768-v-noema.ckpt'
if not os.path.isfile(output):
    gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1oNt0YwJvyNQfrTSr2V3A5ee6AIKqvmLp
From (redirected): https://drive.google.com/uc?id=1oNt0YwJvyNQfrTSr2V3A5ee6AIKqvmLp&confirm=t&uuid=4975f063-c551-40a6-a340-94c8a8d1fdcb
To: /content/STTR2022/checkpoint_model/checkpoint0005.pth
100%|██████████| 545M/545M [00:08<00:00, 66.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1pWOLw6ZiQJq0KkQyrZwNKeBVqEhWaamo
From (redirected): https://drive.google.com/uc?id=1pWOLw6ZiQJq0KkQyrZwNKeBVqEhWaamo&confirm=t&uuid=5e5fbbd3-aa77-4d87-acab-ea4d580d1780
To: /content/stablediffusion2022/checkpoints/768-v-noema.ckpt
100%|██████████| 5.21G/5.21G [01:03<00:00, 82.0MB/s]


# **Import package**

In [ ]:
import sys
sys.path.append('/content/STTR2022')
sys.path.append('/content/stablediffusion2022')
sys.path.append('/content/nghiencuukhoahoc')
import shutil
import gc
import os
from PIL import Image
from colabcode import ColabCode
from typing import List
from fastapi import FastAPI, File, Form, UploadFile, Request
from fastapi.responses import HTMLResponse, FileResponse
from fastapi.templating import Jinja2Templates
from pyngrok import ngrok
import torch
import argparse
from STTR2022.run import get_args_parser, main as main1
from stablediffusion2022.txt2img import main as main2, parse_args as parse_args2
from stablediffusion2022.img2img import main as main3, parse_args as parse_args3

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# **App**

In [ ]:
ngrok.set_auth_token('2OGTbUuflDSL4YMX5ruu8tqA7FK_3eiMwLxKnM76H7RMt6dCG')
app = FastAPI(title="FastAPI on Google Colab", description="FastAPI and ColabCode", version="1.0")

templates = Jinja2Templates(directory='./nghiencuukhoahoc/giaodien_nghiencuu/')


@app.get('/')
def index(request: Request):
    return templates.TemplateResponse('index.html', {'request': request})
@app.get('/style_images', response_class=HTMLResponse)
def style_images_web(request: Request):
    return templates.TemplateResponse('style_images.html', {'request': request})
@app.get('/text_to_images', response_class=HTMLResponse)
def text_to_images_web(request: Request):
    return templates.TemplateResponse('text_to_images.html', {'request': request})
@app.get('/image_to_image', response_class=HTMLResponse)
def image_to_image_web(request: Request):
    return templates.TemplateResponse('image_to_image.html', {'request': request})


# app1
@app.post('/style_images', response_class=HTMLResponse)
async def upload_imgs_to_create_imgs(request: Request,
                                     content_images: List[UploadFile] = File(..., description="Content image (JPG or PNG)"),
                                     style_images: List[UploadFile] = File(..., description="Style image (JPG or PNG)")):
    style_folder = './STTR2022/inputs/style'
    if not os.path.exists(style_folder):
        os.makedirs(style_folder)
    else:
        shutil.rmtree(style_folder)
        os.makedirs(style_folder)

    content_folder = './STTR2022/inputs/content'
    if not os.path.exists(content_folder):
        os.makedirs(content_folder)
    else:
        shutil.rmtree(content_folder)
        os.makedirs(content_folder)

    outputs_folder = './STTR2022/outputs'
    if os.path.exists(outputs_folder) and os.listdir(outputs_folder):
        shutil.rmtree(outputs_folder)

    error_messages = []  # Danh sách thông báo lỗi

    # Kiểm tra tệp Content image
    for idx, content_image in enumerate(content_images):
        if content_image.filename == '':
            error_messages.append(f"Content Image {idx+1} is empty.")
        else:
            with open(os.path.join(content_folder, content_image.filename), 'wb') as f:
                shutil.copyfileobj(content_image.file, f)

    # Kiểm tra tệp Style image
    for idx, style_image in enumerate(style_images):
        if style_image.filename == '':
            error_messages.append(f"Style Image {idx+1} is empty.")
        else:
            with open(os.path.join(style_folder, style_image.filename), 'wb') as f:
                shutil.copyfileobj(style_image.file, f)

    # Kiểm tra nếu có thông báo lỗi, trả về template với thông báo lỗi
    if len(error_messages) > 0:
        return templates.TemplateResponse('style_images.html', {'request': request, 'errors': error_messages})

    # Xử lý tạo ảnh từ ảnh ở đây
    parser = argparse.ArgumentParser('DETR outputs', parents=[get_args_parser()])
    args = parser.parse_args([
        "--batch_size", "1",
        "--style_loss_coef", "10",
        "--fold_k", "5",
        "--lr_backbone", "1e-5",
        "--lr", "1e-5",
        "--enc_layers", "6",
        "--dec_layers", "6",
        "--model_type", "nofold",
        "--enorm", "--dnorm", "--tnorm",
        "--cbackbone_layer", "2", "--sbackbone_layer", "4",

        "--dataset_file", "demo",
        "--resume", "./STTR2022/checkpoint_model/checkpoint0005.pth",
        "--img_size", "512",
        "--content_folder", "./STTR2022/inputs/content",
        "--style_folder", "./STTR2022/inputs/style",
        "--output_dir", "./STTR2022/outputs",
    ])

    main1(args)
    torch.cuda.empty_cache()
    gc.collect()
    success = True
    return templates.TemplateResponse('style_images.html', {'request': request, 'success': success})


# app2
@app.post('/text_to_images', response_class=HTMLResponse)
async def create_image_from_text(request: Request,
                                 text: str = Form(default=None, description="Enter text in English")):
    if os.path.exists('stablediffusion2022/outputs'):
        shutil.rmtree('stablediffusion2022/outputs')

    if not text:
        error_message = "Empty text."
        return templates.TemplateResponse('text_to_images.html', {'request': request, 'errors': error_message})
    if not text.isascii():
        error_message = "Enter text in English."
        return templates.TemplateResponse('text_to_images.html', {'request': request, 'errors': error_message})

    # Xử lý tạo ảnh từ văn bản ở đây
    parser = argparse.ArgumentParser(parents=[parse_args2()])
    # text = "photo of car brand mercedes maybach s650"

    opt = parser.parse_args([
        "--prompt", text,
        "--outdir", "./stablediffusion2022/outputs/txt2img/768-v-noema",
        "--ckpt", "./stablediffusion2022/checkpoints/768-v-noema.ckpt",
        "--config", "./stablediffusion2022/configs/stable-diffusion/v2-inference-v.yaml",
        "--n_samples", "1",
        "--n_iter", "1",
        "--H", "768",
        "--W", "768",
        "--device", "cuda",
    ])
    main2(opt)
    torch.cuda.empty_cache()
    gc.collect()
    success = True
    return templates.TemplateResponse('text_to_images.html', {'request': request, 'success': success})


# app3
@app.post('/image_to_image', response_class=HTMLResponse)
async def create_img_from_img_and_text(request: Request,
                                       strength: float = Form(default=0.5, description="Strength value between 0 and 1"),
                                       text: str = Form(default=None, description="Entering text in English will have more accurate results"),
                                       images: List[UploadFile] = File(..., description="Sample images(JPG or PNG)")):
    sample_image_folder = './stablediffusion2022/inputs'
    if not os.path.exists(sample_image_folder):
        os.makedirs(sample_image_folder)
    else:
        shutil.rmtree(sample_image_folder)
        os.makedirs(sample_image_folder)

    if os.path.exists('stablediffusion2022/outputs'):
        shutil.rmtree('stablediffusion2022/outputs')

    if images[0].filename == '':
        error_message = "Empty image."
        return templates.TemplateResponse('image_to_image.html', {'request': request, 'errors': error_message})
    if not text:
        error_message = "Empty text."
        return templates.TemplateResponse('image_to_image.html', {'request': request, 'errors': error_message})

    img = Image.open(images[0].file)
    img_size = img.size

    if img_size[0] < 256 or img_size[1] < 256:
        error_message = f"input image must be larger than 256x256 but your input image size {img_size[0]}x{img_size[1]}"
        return templates.TemplateResponse('image_to_image.html', {'request': request, 'errors': error_message})

    img.save(f"{sample_image_folder}/{images[0].filename}")

    # Xử lý tạo ảnh từ ảnh ở đây
    parser = argparse.ArgumentParser(parents=[parse_args3()])

    input_file = os.path.join(sample_image_folder, os.listdir(sample_image_folder)[0])
    opt = parser.parse_args([
        "--prompt", text,
        "--init_img", input_file,
        "--outdir", "./stablediffusion2022/outputs/img2img/768-v-noema/",
        "--strength", f"{strength}",
        "--n_samples", '2',
        "--ckpt", './stablediffusion2022/checkpoints/768-v-noema.ckpt',
        "--config", "./stablediffusion2022/configs/stable-diffusion/v2-inference-v.yaml",
        "--device", "cuda",
    ])
    main3(opt)
    torch.cuda.empty_cache()
    gc.collect()
    success = True
    return templates.TemplateResponse('image_to_image.html', {'request': request, 'success': success})


# zip1
@app.get('/download_images1')
def get_images_style():
    directory = './STTR2022/outputs/test_outputs'
    if not os.path.exists(directory):
        return templates.TemplateResponse('style_images.html', {'errors': "No output image yet"})
    zip_file = "./STTR2022/outputs1.zip"
    if os.path.exists(zip_file):
        os.remove(zip_file)

    os.system(f'zip -r {zip_file} {directory}')
    return FileResponse(zip_file, media_type='application/zip')


# zip2
@app.get('/download_images2')
def get_images_txt2img():
    directory = './stablediffusion2022/outputs/txt2img/768-v-noema/samples'
    if not os.path.exists(directory):
        return templates.TemplateResponse('text_to_images.html', {'errors': "No output image yet"})
    zip_file = "./stablediffusion2022/outputs2.zip"
    if os.path.exists(zip_file):
        os.remove(zip_file)

    os.system(f'zip -r {zip_file} {directory}')
    return FileResponse(zip_file, media_type='application/zip')


# zip3
@app.get('/download_images3')
def get_images_img2img():
    directory = './stablediffusion2022/outputs/img2img/768-v-noema/samples'
    if not os.path.exists(directory):
        return templates.TemplateResponse('image_to_image.html', {'errors': "No output image yet"})
    zip_file = './stablediffusion2022/outputs3.zip'
    if os.path.exists(zip_file):
        os.remove(zip_file)

    os.system(f'zip -r {zip_file} {directory}')
    return FileResponse(zip_file, media_type='application/zip')


# **main**

In [ ]:
if __name__ == '__main__':
    cc = ColabCode(port=12000, code=False)
    cc.run_app(app=app)

INFO:     Started server process [202]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://0298-34-82-206-198.ngrok-free.app" -> "http://localhost:12000"
INFO:     2405:4802:5e7b:9c80:6c15:32f:2da:7076:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:4802:5e7b:9c80:6c15:32f:2da:7076:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2405:4802:5e7b:9c80:6c15:32f:2da:7076:0 - "GET /text_to_images HTTP/1.1" 200 OK
INFO:     2405:4802:5e7b:9c80:6c15:32f:2da:7076:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:4802:5e7b:9c80:6c15:32f:2da:7076:0 - "GET /text_to_images HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [202]


In [ ]:
!kill -9 -0